In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from gurobipy import *
#from ucimlrepo import fetch_ucirepo

## Housing

In [2]:
df = pd.read_csv('../data/kc_house_data.csv')

In [3]:
df_dm=df.copy()
# just take the year from the date column
df_dm['sales_yr']=df_dm['date'].astype(str).str[:4]

# add the age of the buildings when the houses were sold as a new column
df_dm['age']=df_dm['sales_yr'].astype(int)-df_dm['yr_built']
# add the age of the renovation when the houses were sold as a new column
df_dm['age_rnv']=0
df_dm['age_rnv']=df_dm['sales_yr'][df_dm['yr_renovated']!=0].astype(int)-df_dm['yr_renovated'][df_dm['yr_renovated']!=0]
df_dm['age_rnv'][df_dm['age_rnv'].isnull()]=0

# partition the age into bins
bins = [-2,0,5,10,25,50,75,100,100000]
labels = ['<1','1-5','6-10','11-25','26-50','51-75','76-100','>100']
df_dm['age_binned'] = pd.cut(df_dm['age'], bins=bins, labels=labels)
# partition the age_rnv into bins
bins = [-2,0,5,10,25,50,75,100000]
labels = ['<1','1-5','6-10','11-25','26-50','51-75','>75']
df_dm['age_rnv_binned'] = pd.cut(df_dm['age_rnv'], bins=bins, labels=labels)

# transform the factor values to be able to use in the model
df_dm = pd.get_dummies(df_dm, columns=['age_binned','age_rnv_binned'])

C:\Users\atevs\AppData\Local\Temp\ipykernel_12928\4109102851.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dm['age_rnv'][df_dm['age_rnv'].isnull()]=0


In [4]:
features = ['bedrooms','bathrooms','sqft_living','sqft_basement','grade','zipcode','lat','long','sqft_lot','waterfront',]

### Optimization model

In [5]:
df_lots = pd.DataFrame({"zipcode": [98108, 98011, 98033, 98004, 98053, 98136, 98004, 98019, 98155, 98115],
                       "lat": [47.5521, 47.7834, 47.6939, 47.6132, 47.6573, 47.5532, 47.6215, 47.7361, 47.7494, 47.6899],
                       "lon": [-122.236, -122.229, -122.01, -122.273, -122.143, -122.321, -122.277, -121.928, -122.214, -122.281],
                       "sqftlot": [3480, 5230, 10542, 900, 4560, 7280, 6780, 9873, 2560, 1080],
                       "water": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]})
c_bed = 10000
c_bath = 20000
c_base = 60
c_lot = [250000] * 10
c_grade = [88.45, 98.30, 109.03, 126.59, 148.11, 169.73, 231.25, 330.81, 380.45]
B = 450000

In [6]:
class RFOptimizer:
    def __init__(self, T) -> None:
        self.T = T
        self.rf = RandomForestRegressor(n_estimators=T, random_state=137)

    def loadData(self, df, features, outcome: str):
        self.df = df
        self.outcome = outcome
        self.features = features

    def trainRF(self, tr_size=0.7):
        self.train_data, self.test_data = train_test_split(self.df, train_size = tr_size, random_state=4)
        self.rf.fit(self.train_data[self.features], self.train_data[self.outcome])
        self.trees = self.rf.estimators_ #just a "reference" for convenience

        print(("R-squared (train) {r2:.3f}").format(r2=self.rf.score(self.train_data[self.features], self.train_data[self.outcome])))
        print(("R-squared (trest) {r2:.3f}").format(r2=self.rf.score(self.test_data[self.features], self.test_data[self.outcome])))

    def buildOptModel(self, A, B, c_bed, c_bath, c_base, c_lot, c_grade):
        self.m = Model()
        self.m.params.NonConvex = 2
        M = max(self.df[features].max())

    #Init variables
        #data-specific block
        self.z_grade = self.m.addVars(len(c_grade), vtype=GRB.BINARY, name="z_grade")
        self.u = self.m.addVars(len(A), vtype=GRB.BINARY, name="u_lot")
        self.z_bed = self.m.addVar(vtype=GRB.INTEGER, name="z_bed")
        self.z_bath = self.m.addVar(vtype=GRB.INTEGER, name="z_bath")
        self.z_size = self.m.addVar(vtype=GRB.CONTINUOUS, name="z_size")
        self.z_base = self.m.addVar(vtype=GRB.CONTINUOUS, name="z_base")
        self.ld = self.m.addVars(len(self.train_data), vtype=GRB.CONTINUOUS, name="ld")
        w = [A["zipcode"] * np.array(self.u.values()), A["lat"] * np.array(self.u.values()), A["lon"] * np.array(self.u.values()), 
             A["sqftlot"] * np.array(self.u.values()), A["water"] * np.array(self.u.values())]
        for_splits = [self.z_bed, self.z_bath, self.z_size, self.z_base, np.dot(self.z_grade.values(), range(4, 13)), sum(w[0]), sum(w[1]), sum(w[2]), sum(w[3]), sum(w[4])]

        #RF variables
        self.q = {}
        self.leaf_parents = []#dict containing leaf: parent pairs for each tree
        for t in range(self.T):
            leaf_parentsd = {}
            for i in range(len(self.rf.estimators_[t].tree_.feature)):
                if self.trees[t].tree_.children_left[i] > -1:
                    self.q[t, i, self.trees[t].tree_.children_left[i]] = self.m.addVar(vtype=GRB.BINARY, name="q_%d_%d_%d(l)" % (t, i, self.trees[t].tree_.children_left[i]))
                    self.q[t, i, self.trees[t].tree_.children_right[i]] = self.m.addVar(vtype=GRB.BINARY, name="q_%d_%d_%d(l)" % (t, i, self.trees[t].tree_.children_right[i]))
                    l = self.trees[t].tree_.children_left[i]
                    r = self.trees[t].tree_.children_right[i]
                    if self.trees[t].tree_.feature[l] == -2: #if left child a leaf add to leaves
                        leaf_parentsd[l] = i
                    if self.trees[t].tree_.feature[r] == -2: #if right child a leaf add to leaves
                        leaf_parentsd[r] = i
            self.leaf_parents.append(leaf_parentsd)
    
        self.m.update()

        #RF structure constraints
        for t in range(self.T):
            for i in range(len(self.rf.estimators_[t].tree_.feature)):
                x = self.trees[t].tree_.feature[i]
                b = self.trees[t].tree_.value[i][0][0]
                if x > -2:
                    l = self.trees[t].tree_.children_left[i]
                    r = self.trees[t].tree_.children_right[i]
                    self.m.addConstr(for_splits[x] - 10*M*(1 - self.q[t, i, l]) <= b) #assuming left is true
                    self.m.addConstr(for_splits[x] + 10*M*(1 - self.q[t, i, r]) >= b) #assuming left is true
                    if self.trees[t].tree_.children_right[l] > -1:
                        self.m.addConstr(self.q[t, l, self.trees[t].tree_.children_left[l]] + self.q[t, l, self.trees[t].tree_.children_right[l]] == self.q[t, i, l])
                    if self.trees[t].tree_.children_right[r] > -1:
                        self.m.addConstr(self.q[t, r, self.trees[t].tree_.children_left[r]] + self.q[t, r, self.trees[t].tree_.children_right[r]] == self.q[t, i, r])
            self.m.addConstr(quicksum(self.q[t, self.leaf_parents[t][i], i] for i in self.leaf_parents[t].keys()) == 1)

        #application constraints
        self.m.addConstr(self.u.sum() == 1)
        self.m.addConstr(c_bed * self.z_bed + c_bath * self.z_bath + c_base * self.z_base + np.dot(c_lot, self.u.values()) 
                         + self.z_size * np.dot(c_grade, self.z_grade.values()) <= B)
        self.m.addConstr(self.z_size <= for_splits[8])
        self.m.addConstr(self.z_grade.sum() == 1)
        self.m.addConstr(self.z_bed == np.dot(self.ld.values(), self.train_data["bedrooms"]))
        self.m.addConstr(self.z_bath == np.dot(self.ld.values(), self.train_data["bathrooms"]))
        self.m.addConstr(self.z_size == np.dot(self.ld.values(), self.train_data["sqft_living"]))
        self.m.addConstr(self.z_base == np.dot(self.ld.values(), self.train_data["sqft_basement"]))
        self.m.addConstr(self.ld.sum() == 1)

        self.m.setObjective(1/self.T * quicksum(self.trees[t].tree_.value[i][0][0] * self.q[t, self.leaf_parents[t][i], i] for t in range(self.T) for i in self.leaf_parents[t].keys()), GRB.MAXIMIZE)

    def solve(self):
        self.m.optimize()

    def printSol(self):
        print("Buying lot %d" % sum([self.u[i].x * i for i in range(len(self.u))]))
        print("Bedrooms: %g" % self.z_bed.x)
        print("Bathrooms: %g" % self.z_bath.x)
        print("Basement size: %g" % self.z_base.x)
        print("Sqft living: %g" % self.z_size.x)
        print("Grade: %d" % sum([self.z_grade[i].x * (i + 4) for i in range(len(self.z_grade))]))
        for i in range(len(self.train_data)):
            if self.ld[i].x > 0.001:
                print(self.train_data.iloc[i])
                print(self.ld[i].x)

In [7]:
optRF = RFOptimizer(10)
optRF.loadData(df_dm, features, "price")

In [8]:
optRF.trainRF(tr_size=0.7)

R-squared (train) 0.974
R-squared (trest) 0.848


In [9]:
optRF.buildOptModel(df_lots, B, c_bed, c_bath, c_base, c_lot, c_grade)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-10-08
Set parameter NonConvex to value 2


In [10]:
optRF.solve()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Xeon(R) W-10855M CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 278537 rows, 200838 columns and 1852572 nonzeros
Model fingerprint: 0x506f91f8
Model has 1 quadratic constraint
Variable types: 15131 continuous, 185707 integer (185705 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+07]
  QMatrix range    [9e+01, 4e+02]
  QLMatrix range   [6e+01, 3e+05]
  Objective range  [8e+03, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+07]
  QRHS range       [5e+05, 5e+05]
Presolve removed 278529 rows and 191720 columns
Presolve time: 0.66s
Presolved: 18 rows, 9127 columns, 42162 nonzeros
Variable types: 9106 continuous, 21 integer (19 binary)
Found heuristic solution: objective 275730.00000

Explored 0 nodes (0 simplex iterations) in 0.78 seconds (0.55 work units)
Thread count was 12 

In [11]:
optRF.printSol()

Buying lot 3
Bedrooms: -0
Bathrooms: -0
Basement size: 0
Sqft living: 844
Grade: 4
id                           9543000205
date                    20150413T000000
price                          139950.0
bedrooms                              0
bathrooms                           0.0
sqft_living                         844
sqft_lot                           4269
floors                              1.0
waterfront                            0
view                                  0
condition                             4
grade                                 7
sqft_above                          844
sqft_basement                         0
yr_built                           1913
yr_renovated                          0
zipcode                           98001
lat                             47.2781
long                            -122.25
sqft_living15                      1380
sqft_lot15                         9600
sales_yr                           2015
age                                 1